In [1]:
# import ee and geemap define map
import ee 
import geemap
import math
geemap.ee_initialize()
Map = geemap.Map()

In [2]:
# generate transverse polygons
def hexGrid(proj, diameter):
  size = ee.Number(diameter).divide(math.sqrt(3)) # Distance from center to vertex

  coords = ee.Image.pixelCoordinates(proj)
  vals = {
    # Switch x and y here to get flat top instead of pointy top hexagons.
    'x': coords.select("x"),
    'u': coords.select("x").divide(diameter),  # term 1
    'v': coords.select("y").divide(size),      # term 2
    'r': ee.Number(diameter).divide(2),
  }
  i = ee.Image().expression("floor((floor(u - v) + floor(x / r))/3)", vals)
  j = ee.Image().expression("floor((floor(u + v) + floor(v - u))/3)", vals)

  # Turn the hex coordinates into a single "ID" number.
  cells = i.long().leftShift(32).add(j.long()).rename("hexgrid")
  return cells

In [3]:
# Define a map ROI
roi = Map.user_roi

if roi is None:
    roi = ee.Geometry.Polygon(
        [
            [
                [-64.61216757645253,-27.607649211785315],
                [-58.196151951452535,-27.607649211785315],
                [-58.196151951452535,-17.897238844252453],
                [-64.61216757645253,-17.897238844252453],
                [-64.61216757645253,-27.607649211785315],
            ]
        ]
    )
    Map.addLayer(roi, {}, 'ROI')
Map

# Generate a hex grid and mask off cells that don't touch the region.
grid = hexGrid(ee.Projection('EPSG:5070'), 70000)
regionImg = ee.Image(0).byte().paint(roi, 1)
mask = grid.addBands(regionImg) \
    .reduceConnectedComponents(ee.Reducer.max(), "hexgrid", 256)
grid = grid.updateMask(mask)
Map.addLayer(grid, {}, "hexagons")

In [4]:
# Add Hansen
hansen = ee.Image('UMD/hansen/global_forest_change_2021_v1_9')
treeCoverVisParam = {
  'bands': ['treecover2000'],
  'min': 0,
  'max': 100,
  'palette': ['black', 'green']
}
Map.addLayer(hansen, treeCoverVisParam, 'tree cover', False)

treeLossVisParam = {
  'bands': ['lossyear'],
  'min': 0,
  'max': 21,
  'palette': ['yellow', 'red']
}
Map.addLayer(hansen, treeLossVisParam, 'tree loss year', False)

In [5]:
# Clip Hansen to ROI
hansen_clip = hansen.clip(roi)
Map.addLayer(hansen_clip, treeCoverVisParam, 'tree cover clip')
Map.addLayer(hansen_clip, treeLossVisParam, 'tree loss year clip')
Map.centerObject(roi,6)
Map

Map(center=[-22.72646711982983, -61.404159763952485], controls=(WidgetControl(options=['position', 'transparen…

In [12]:
# Calculalate mean tree loss per hexagon
mean_loss = hansen_clip.reduceConnectedComponents(ee.Reducer.mean())

meanlossVisParam = {
  'bands': ['lossyear'],
  'min': 0,
  'max': 21,
  'palette': ['440154', '433982', '30678D', '218F8B', '36B677', '8ED542', 'FDE725', 'red']
}
Map.addLayer(mean_loss, meanlossVisParam, "mean loss")
Map.add_colorbar(meanlossVisParam, label="Mean Tree Loss", layer_name="mean loss")
Map

Map(bottom=9553.0, center=[-22.695120184965695, -56.86523437500001], controls=(WidgetControl(options=['positio…

In [7]:
# out_dem_stats = 'chaco_loss_stats.csv'
# geemap.zonal_stats(mean_loss, roi, out_dem_stats, scale=1000, return_fc=False)